In [1]:
%%capture
!pip install transformers
!pip install pypinyin
!pip install jieba
!pip install paddlepaddle

In [11]:
import re,time,json
from collections import defaultdict
from torch.utils.data import DataLoader
from pypinyin import pinyin, Style
from tqdm import tqdm
import pickle

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from copy import deepcopy

from transformers import (BertTokenizer,BertConfig,BertModel)

from model.Embedding import *
from model.fusionDataset import FusionDataset

import jieba
import jieba.posseg as pseg
import paddle

config = BertConfig.from_pretrained('AnchiBERT')
tokenizer = BertTokenizer.from_pretrained('AnchiBERT')
Anchibert = BertModel.from_pretrained('AnchiBERT',config=config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at AnchiBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Load Necessary preproceeded Data

In [3]:
with open('data/char_map.json','r') as f:
    glyph2ix = defaultdict(lambda : 1)
    glyph2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    glyph2ix.update(json.load(f))

with open('data/pinyin_map.json','r') as f:
    pinyin2ix = defaultdict(lambda : 1)
    pinyin2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    pinyin2ix.update(json.load(f))
    
with open('data/pos_tags.json','r') as f:
    pos2ix = defaultdict(lambda : 0)
    pos2ix.update(json.load(f))

In [4]:
# train 上联
with open("couplet/train/in.txt",encoding='utf8') as f:
    tr_in =  [row.strip().split() for row in f.readlines()]
# train 下联  
with open("couplet/train/out.txt",encoding='utf8') as f:
    tr_out = [row.strip().split() for row in f.readlines()]
with open('data/train_in_pos.pt','rb') as f:
    tr_pos_in = pickle.load(f)
with open('data/train_out_pos.pt','rb') as f:
    tr_pos_out = pickle.load(f)

In [3]:
with open("couplet/test/in.txt",encoding='utf8') as f:
    te_in =  [row.strip().split() for row in f.readlines()]
# train 下联  
with open("couplet/test/out.txt",encoding='utf8') as f:
    te_out = [row.strip().split() for row in f.readlines()]

In [12]:
display(len(tr_in))
total_len = len(tr_in)
half = total_len//7
display(half)
train_split = int(0.80 * half)


770491

110070

In [13]:
trainSet = FusionDataset(tr_in[:train_split],tokenizer,\
                         glyph2ix,pinyin2ix,pos2ix,tr_out[:train_split],\
                         tr_pos_in[:train_split],tr_pos_out[:train_split],\
                         device=device) # use device if you want to load it gpu

88056it [00:39, 2237.35it/s]
88056it [00:41, 2112.32it/s]


In [14]:
valSet = FusionDataset(tr_in[train_split:half],tokenizer,\
                       glyph2ix,pinyin2ix,pos2ix,tr_out[train_split:half],\
                       tr_pos_in[train_split:half],tr_pos_out[train_split:half],\
                       device=device) # use device if you want to load it gpu

22014it [00:10, 2168.63it/s]
22014it [00:10, 2078.53it/s]


In [8]:
from model.fusion_transformer import Fusion_Anchi_Trans_Decoder, Fusion_Anchi_Transformer, Anchi_Decoder,Anchi_Transformer

In [9]:
from utils.trans_trainer import train

## Fusion_Anchi_Trans_Decoder

In [15]:
# for i in range(1,6):
config = { # for Fusion_Anchi_Trans_Decoder
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30, # trainable
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10, # trainable
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6 , #6, trainable
    'output_dim':21128,# fixed
    'device':device,
}
# batch_size = [32,64,128]
# lr =[0.1,0.01,0.001]
# <model_name>_<optim>_<batch_num>_<lr>_<epoch>_<num_layer>_<pinyin_embed_dim>_<tag_emb_dim>_<train_size>
name = f'fu_anchi_de_Adam_128_0001_60_6_30_10_110k'
train(Fusion_Anchi_Trans_Decoder(config),trainSet,valSet,batch_size=128,lr=0.0001,
      epoch=60,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/60 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 5m 51s
	Training Loss: 5.35195 	Validation Loss: 4.16573


  2%|▏         | 1/60 [05:53<5:47:12, 353.10s/it]

Epoch: 02 | Epoch Time: 5m 54s
	Training Loss: 2.89384 	Validation Loss: 2.60486


  3%|▎         | 2/60 [11:48<5:42:39, 354.48s/it]

...

Epoch: 23 | Epoch Time: 5m 52s
	Training Loss: 0.07298 	Validation Loss: 1.19600


 40%|████      | 24/60 [2:21:39<3:31:54, 353.17s/it]

Epoch: 24 | Epoch Time: 5m 53s
	Training Loss: 0.06817 	Validation Loss: 1.22152


 40%|████      | 24/60 [2:27:33<3:41:20, 368.89s/it]

Epoch: 25 | Epoch Time: 5m 53s
	Training Loss: 0.06407 	Validation Loss: 1.26311


## Anchi_Decoder

In [ ]:
config = { # for Trans_Decoder
    'max_position_embeddings':50,
    'hidden_size':768,
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6, # trainable
    'output_dim':21128, # fixed
    'device':device
}
# <model_name>_<optim>_<batch_num>_<lr>_<epoch>_<num_layer>_<train_size>
name = 'anchi_de_Adam_128_0001_10_60_6_110k'
train(Anchi_Decoder(config),trainSet,valSet,batch_size=128,lr=0.0001,
      epoch=60,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/60 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 5m 43s
	Training Loss: 5.28061 	Validation Loss: 4.09259


  2%|▏         | 1/60 [05:44<5:39:02, 344.79s/it]

Epoch: 02 | Epoch Time: 5m 45s
	Training Loss: 2.94924 	Validation Loss: 2.67170


  3%|▎         | 2/60 [11:31<5:34:20, 345.88s/it]

...

 15%|█▌        | 9/60 [51:57<4:54:32, 346.52s/it]

Epoch: 10 | Epoch Time: 5m 45s
	Training Loss: 0.66106 	Validation Loss: 1.33461


 17%|█▋        | 10/60 [57:44<4:48:50, 346.61s/it]

Epoch: 11 | Epoch Time: 5m 45s
	Training Loss: 0.58239 	Validation Loss: 1.31621


 18%|█▊        | 11/60 [1:03:31<4:43:07, 346.68s/it]

## Fusion_Anchi_Transformer

In [13]:
config = { # Fusion_Anchi_Transformer
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30, # trainable
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10, # trainable 
    'layer_norm_eps':1e-12, 
    'hidden_dropout':0.1, 
    'nhead':12,
    'num_encoder_layers':5, # trainable
    'num_decoder_layers':6, # trainable
    'output_dim':21128, # fixed
    'dim_feedforward': 3072,
    'activation':'relu',
    'trans_dropout':0.1,
    'device':device
}
# <model_name>_<optim>_<batch_num>_<lr>_<epoch>_<pinyin_embed_dim>_<tag_emb_dim>_<encoder layer>_<decoder layer>
name = 'fu_anchi_tra_Adam_128_01_60_30_10_5_6'
train(Fusion_Anchi_Transformer(config),trainSet,valSet,batch_size=128,lr=0.01,
      epoch=60,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

  0%|          | 0/60 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 3m 33s
	Training Loss: 6.89782 	Validation Loss: 6.63895


  2%|▏         | 1/60 [03:34<3:31:14, 214.83s/it]

Epoch: 02 | Epoch Time: 3m 34s
	Training Loss: 6.59584 	Validation Loss: 6.61068


  3%|▎         | 2/60 [07:11<3:28:29, 215.68s/it]

...

Epoch: 27 | Epoch Time: 3m 34s
	Training Loss: 6.55731 	Validation Loss: 6.56183


 47%|████▋     | 28/60 [1:40:22<1:54:34, 214.83s/it]

Epoch: 28 | Epoch Time: 3m 34s
	Training Loss: 6.55844 	Validation Loss: 6.56411


 47%|████▋     | 28/60 [1:43:57<1:58:48, 222.76s/it]

Epoch: 29 | Epoch Time: 3m 34s
	Training Loss: 6.55779 	Validation Loss: 6.56255


## Anchi_Transformer

In [ ]:
config = { # Anchi_Transformer
    'max_position_embeddings':50,
    'hidden_size':768,
    'layer_norm_eps':1e-12, 
    'hidden_dropout':0.1, 
    'nhead':12,
    'num_encoder_layers':6, # trainable
    'num_decoder_layers':6, # trainable
    'output_dim':21128, # fixed
    'dim_feedforward': 3072,
    'activation':'relu',
    'trans_dropout':0.1,
    'device':device
}
# <model_name>_<optim>_<batch_num>_<lr>_<epoch>_<encoder layer>_<decoder layer>
name = 'anchi_tra_Adam_128_01_60_6_6'
train(Anchi_Transformer(config),trainSet,valSet,batch_size=128,lr=0.01,
      epoch=60,bert=Anchibert,name= name, with_trans=True,
      optimizer_name='Adam',scheduleFactor=0.5,
      schedule_Patience=5,min_lr=1e-06,verbose=True
      ,patience=10,store='result/')
torch.cuda.empty_cache()

# Decoder Section

In [5]:
config = { # for Fusion_Anchi_Trans_Decoder
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30, # trainable
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10, # trainable
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6 , #6, trainable
    'output_dim':21128,# fixed
    'device':device,
}
model = Fusion_Anchi_Trans_Decoder(config)
model.load_state_dict(torch.load('result/fu_anchi_de_Adam_128_0001_60_6_30_10_110k.pt'))

<All keys matched successfully>

In [6]:
from utils.generate_couplet import greedy_decode

In [ ]:
%%capture
predict = greedy_decode(model=model,
                      bert=Anchibert,
                      tokenizer=tokenizer,
                      sent=te_in[0],
                      glyph2ix=glyph2ix,
                      pinyin2ix=pinyin2ix,
                      pos2ix=pos2ix,device=device)

In [11]:
' '.join(te_in[0]),' '.join(predict)

('腾 飞 上 铁 ， 锐 意 改 革 谋 发 展 ， 勇 当 千 里 马', '自 由 自 由 ， 自 由 自 由 自 由 自 由 自 由 自 由 自')